In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D , Flatten, Activation, Dropout, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
import numpy as np
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

In [30]:
generator = ImageDataGenerator(validation_split= 0.2, rescale=1./255, horizontal_flip= True, vertical_flip= True, rotation_range= 10, height_shift_range= 0.2, width_shift_range= 0.2)

In [2]:
img = image.load_img(r"C:\Users\gonza\Downloads\PokemonData\Zubat\3e91e56b8ea045a5b2db1b3eea58cfac.jpg", target_size= (224,224))
x = image.img_to_array(img)
x = x.reshape((1,) + x.shape)

i = 0
for batch in generator.flow(x, batch_size= 1, save_to_dir=r'C:\Users\gonza\Downloads\PokemonData\Zubat', save_prefix='Zubat', save_format='jpeg'):

    i += 1
    if i > 20:
        break  # otherwise the generator would loop indefinitely


#traindata = generator.flow_from_directory(directory= r"C:\Users\gonza\Downloads\PokemonData", target_size=(224,224), subset = "training", seed= 66, keep_aspect_ratio= True, color_mode= "rgb")
#testdata = generator.flow_from_directory(directory=r"C:\Users\gonza\Downloads\PokemonData", target_size=(224,224), subset = "validation", seed = 66, keep_aspect_ratio= True, color_mode= "rgb")
#traindata.image_shape
#traindata.class_indices
#np.fromiter(traindata.class_indices.keys(), dtype = str,)
#culo = np.array(list(traindata.class_indices.keys()), dtype= str)



In [31]:
model = Sequential()
model.add(Conv2D(64,padding="same", input_shape=(224, 224, 3), activation="relu", kernel_size= 3))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Conv2D(32,activation="relu", kernel_size= 3))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(150, activation="softmax"))


In [32]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 224, 224, 64)      1792      
                                                                 
 batch_normalization_6 (Bat  (None, 224, 224, 64)      256       
 chNormalization)                                                
                                                                 
 activation_6 (Activation)   (None, 224, 224, 64)      0         
                                                                 
 conv2d_7 (Conv2D)           (None, 222, 222, 32)      18464     
                                                                 
 batch_normalization_7 (Bat  (None, 222, 222, 32)      128       
 chNormalization)                                                
                                                                 
 activation_7 (Activation)   (None, 222, 222, 32)     

Total params: 59161590 (225.68 MB)
Trainable params: 59161398 (225.68 MB)
Non-trainable params: 192 (768.00 Byte)
_________________________________________________________________


In [33]:
traindata = generator.flow_from_directory(directory= r"C:\Users\gonza\Downloads\PokemonData", target_size=(224,224), subset = "training", seed= 66, keep_aspect_ratio= True, color_mode= "rgb")
testdata = generator.flow_from_directory(directory=r"C:\Users\gonza\Downloads\PokemonData", target_size=(224,224), subset = "validation", seed = 66, keep_aspect_ratio= True, color_mode= "rgb")

#np.fromiter(traindata.class_indices.keys(), dtype = str,)
culo = np.array(list(traindata.class_indices.keys()), dtype= str)
traindata.batch_size



Found 5511 images belonging to 150 classes.
Found 1309 images belonging to 150 classes.


32

In [34]:
model.fit(
        traindata,
        epochs= 10,
        validation_data= testdata
        , shuffle= True)
model.save_weights('first_try.h5')  # always save your weights after training or during training

Epoch 1/10
173/173 [==============================] - 267s 2s/step - loss: 120.1968 - accuracy: 0.0256 - val_loss: 16.1243 - val_accuracy: 0.0069
Epoch 2/10
173/173 [==============================] - 265s 2s/step - loss: 48.1224 - accuracy: 0.0385 - val_loss: 8.9325 - val_accuracy: 0.0298
Epoch 3/10
173/173 [==============================] - 265s 2s/step - loss: 19.2479 - accuracy: 0.0537 - val_loss: 21.5602 - val_accuracy: 0.0252
Epoch 4/10
173/173 [==============================] - 265s 2s/step - loss: 5.6856 - accuracy: 0.0726 - val_loss: 7.7886 - val_accuracy: 0.0367
Epoch 5/10
173/173 [==============================] - 266s 2s/step - loss: 4.2193 - accuracy: 0.1129 - val_loss: 5.2035 - val_accuracy: 0.0657
Epoch 6/10
173/173 [==============================] - 265s 2s/step - loss: 3.8365 - accuracy: 0.1354 - val_loss: 4.4445 - val_accuracy: 0.1345
Epoch 7/10
173/173 [==============================] - 265s 2s/step - loss: 3.6152 - accuracy: 0.1617 - val_loss: 4.4151 - val_accuracy: 

https://builtin.com/machine-learning/vgg16

https://towardsdatascience.com/transfer-learning-with-vgg16-and-keras-50ea161580b4